In [2]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [1]:
import warnings
warnings.filterwarnings(action="ignore")

# 웹수집 방법
- 웹스크랩핑 : requests, urllib, bs4, selenium => (http, css 지식 필요)
- 공공데이타 포털 제공 => csv, excel
- API 서비스 => 네이버, 카카오, 통계청, 공공데이타포털 (xml, json 지식 필요)
- 판다스 금융라이브러리 (주식, 비트코인, 환율, 원자재....)

# 웹데이타 수집 과정

- 웹에 필요한 URL 요청 (관련 모듈 => requests, urllib)
- response 객체 => html 페이지
- 파싱(bs4, selenium)
- 리스트, 딕셔너리, 판다스의 데이타프레임 형태로 저장
- csv 파일로 저장하거나 데이타베이스에 저장


### requests 모듈을 이용하여 특정 url 주소의 페이지 저장하기

In [5]:
# pip list

In [17]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [14]:
# 요청 => response 객체 생성
url = 'http://google.com'
res = requests.get(url)

# 요청 확인
# res.headers
print(res.headers)

# response 객체 => 텍스트화
print(f'res = {res}')
# res = <Response [200]>
html_str = res.text
print('='*50)
html_str

{'Date': 'Thu, 18 Nov 2021 06:01:36 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=ISO-8859-1', 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'Content-Length': '6526', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': '1P_JAR=2021-11-18-06; expires=Sat, 18-Dec-2021 06:01:36 GMT; path=/; domain=.google.com; Secure, NID=511=VJ00ag7butZI0bNIt4Ck8tzIIMjwPr1xKZsZQKJfN7AqOmsQ_9WLnmF9P8pSs8ha4x8QyoadUFFMTNfx59o-v3L17urL4JW1HX_fyuz2OwUIzyIgtA2qxcFyXKHei99B7ot1AoayLkFv1iHPHuh8vXUUzZA_8g3zbAdyF7_JUZo; expires=Fri, 20-May-2022 06:01:36 GMT; path=/; domain=.google.com; HttpOnly'}
res = <Response [200]>


'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="azrDovWGoBWxm0CzA53oNw==">(function(){window.google={kEI:\'QOyVYbL7EqXemAXN0YHgDA\',kEXPI:\'0,202371,1100165,56873,6059,206,4804,2316,383,246,5,1354,5250,1122516,1197792,609,328866,51224,16114,17444,11240,17572,1326,3533,1361,9290,3022,17587,4020,978,13228,3847,4192,6430,21823,918,5081,885,708,1279,2212,530,149,1103,840,2197,4100,3514,606,2023,1777,520,1704,12966,3227,2847,5,12354,5096,11625,4696,907,2,941,2614,3783,9359,3,346,230,6460,148,12314,1661,4,1528,2304,7039,4684,2014,13611,4764,2658,6701,655,31,13628,1590,715,2132,16786,2521,3294,2542,4094,3138,6,908,3,3541,1,16524,283,912,5998,18437,2,3037,10985,1998,3844,1678,743,5853,9454,1009,1160,1267,5432,2381,2719,14170,4063,27,2,6,7754,4569,2577,3410,265,

In [12]:
# 폴더 생성 후 html 페이지 저장
try:
    os.mkdir('output2')
except:
    pass

with open('output2/google.html', 'w') as f:
    f.write(html_str)

In [15]:
ls output2

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: F424-1196

 C:\Users\User\Desktop\Code\20211118 Python Lib 4일차\output2 디렉터리

2021-11-18  오후 02:56    <DIR>          .
2021-11-18  오후 02:56    <DIR>          ..
2021-11-18  오후 02:58            15,241 google.html
               1개 파일              15,241 바이트
               2개 디렉터리  435,353,239,552 바이트 남음


## bs4를 이용해 파싱 처리하기

## 파싱(Parsing)

가공되지 않은 데이타에서 원하는 정보를 추출하는 과정

## BeautifulSoup
- 외부 라이브러리
- pip list 명령으로 설치 확인 : beautifulsoup4
- 없다면 설치 : pip install beautifulsoup4
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.html


## 뷰티풀숩 임포트
from bs4 import BeautifulSoup

### BeautifulSoup을 이용한 작업 과정
- requests를 이용해서 url 요청 => response 객체 => html화 (텍스트)
- html화된 자료 => 뷰티풀숩 객체화
- 뷰티풀숩의 다양한 함수를 이용해서 자료변환(리스트나 딕셔너리 데이타프레임등)
- csv 파일등로 저장 (판다스를 이용)

### 샘플 스크래핑 url 지정 
url = 'http://pythonscraping.com/pages/warandpeace.html'

In [18]:
# 요청 => response 객체 생성
url = 'http://pythonscraping.com/pages/warandpeace.html'
res = requests.get(url)

# response 객체 => 텍스트화
html_str = res.text
print('='*50)
html_str

'<html>\n<head>\n<style>\n.green{\n\tcolor:#55ff55;\n}\n.red{\n\tcolor:#ff5555;\n}\n#text{\n\twidth:50%;\n}\n</style>\n</head>\n<body>\n<h1>War and Peace</h1>\n<h2>Chapter 1</h2>\n<div id="text">\n"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don\'t tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy \'faithful slave,\' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news.</span>"\n<p/>\nIt was in July, 1805, and the speaker was the well-known <span class="green">Anna\nPavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya\nFedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man\nof high rank

### html화 된 자료 => 뷰티풀숩 객체화
- BeautifulSoup(html변수 , 해석기 )
- 해석기 : html.parser/html5/xml/lxml
- soup = BeautifulSoup(html_str, 'html.parser' )

In [19]:
soup = BeautifulSoup(html_str, 'html.parser')
soup

<html>
<head>
<style>
.green{
	color:#55ff55;
}
.red{
	color:#ff5555;
}
#text{
	width:50%;
}
</style>
</head>
<body>
<h1>War and Peace</h1>
<h2>Chapter 1</h2>
<div id="text">
"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>"
<p></p>
It was in July, 1805, and the speaker was the well-known <span class="green">Anna
Pavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya
Fedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man
of high rank and importance, who was the firs

### bs4의 메서드 이용하기

#### find_all(태그) => 리스트 형태
- soup.find_all('span')

#### find(태그)
- 태그가 여러개일 경우에는 첫번째 태그만 반환 
- soup.find('p')

#### find(id=아이디명)
- soup.find(id='text').text

#### find(class_=클래스명)
- soup.find(class_='green')

#### 특정 태그의 속성값 출력
- get(속성명)

In [24]:
# 특정 태그 한개 정보 추출
print(soup.find('h1'))

# 삽입된 텍스트만 => .text
print(soup.find('h1').text)

<h1>War and Peace</h1>
War and Peace


In [26]:
# 모든 span 태그 => 리스트
spanList = soup.find_all('span')
len(spanList)

75

In [28]:
spanList[5]

<span class="green">St. Petersburg</span>

In [31]:
# 텍스트만 리스트로 저장
spanTextList = []
for item in spanList:
    spanTextList.append(item.text)
    print(item.text)

Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.
Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
If you have nothing better to do, Count [or Prince], and if the
prospect of spending an evening with a poor invalid is not too
terrible, I shall be very charmed to see you tonight between 7 and 10-
Annette Scherer.
Heavens! what a virulent attack!
the prince
Anna Pavlovna
First of all, dear friend, tell me how you are. Set your friend's
mind at rest,
Can one be well while suffering morally? Can one be calm in times
like these if one

In [32]:
spanTextList

["Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don't tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy 'faithful slave,' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news.",
 'Anna\nPavlovna Scherer',
 'Empress Marya\nFedorovna',
 'Prince Vasili Kuragin',
 'Anna Pavlovna',
 'St. Petersburg',
 'If you have nothing better to do, Count [or Prince], and if the\nprospect of spending an evening with a poor invalid is not too\nterrible, I shall be very charmed to see you tonight between 7 and 10-\nAnnette Scherer.',
 'Heavens! what a virulent attack!',
 'the prince',
 'Anna Pavlovna',
 "First of all, dear friend, tell me how you are. Set your friend's\nmind at rest,",
 'Can one be well while suf

In [33]:
spanTextList[:3]

["Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don't tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy 'faithful slave,' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news.",
 'Anna\nPavlovna Scherer',
 'Empress Marya\nFedorovna']

In [36]:
# find_all(class_=클래스명)
# green 클래스가 적용된 태그 => 리스트 
greenList = soup.find_all(class_='green')
len(greenList)

41

In [37]:
greenList[0]

<span class="green">Anna
Pavlovna Scherer</span>

### select() 와 select_one()

- select(선택자) => 리스트
- select_one(선택자) => 1개의 요소

In [38]:
# red 클래스가 적용된 태그 => 리스트 
redList = soup.select('.red')
len(redList)

34

In [39]:
redOne = soup.select_one('.red')
redOne

<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>

#### 속성값 추출
- get(속성명)
- ['속성명']

In [44]:
redOne.text

"Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don't tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy 'faithful slave,' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news."

In [45]:
redOne.get('class')

['red']

In [46]:
redOne['class']

['red']

# 예제1
알렉사 사이트의 50위
https://www.alexa.com/topsites/countries/KR

In [48]:
# url 요청 => response 객체 생성 
url = 'https://www.alexa.com/topsites/countries/KR'
res = requests.get(url)

# response 객체 => 텍스트화 
html_str = res.text
# html_str

In [49]:
# soup 객체화
soup = BeautifulSoup(html_str, 'html.parser')
# soup

In [52]:
# 리스트로 저장
div_list = soup.find_all(class_='tr site-listing')
len(div_list)

50

In [51]:
div_list[-1]

<div class="tr site-listing">
<div class="td">50</div>
<div class="td DescriptionCell">
<p class="">
<a href="/siteinfo/ebay.com">Ebay.com</a>
</p>
</div>
<div class="td right"><p>9:60</p></div>
<div class="td right"><p>7.80</p></div>
<div class="td right"><p>16.70%</p></div>
<div class="td right"><p>81,753</p></div>
</div>

In [59]:
# 순위
print(div_list[-1].select_one('.td').text)
# 사이트 제목
print(div_list[-1].find('a').text)
# 사이트 URL
print(div_list[-1].find('a')['href'])
# daily_time
print(div_list[-1].find_all(class_='td right')[0].text)
# daily_pageviews
print(div_list[-1].find_all(class_='td right')[1].text)

50
Ebay.com
/siteinfo/ebay.com
9:60
7.80


In [61]:
# 2차원 리스트 저장
temp_list = []
for row in div_list:
    rank = row.select_one('.td').text
    title = row.find('a').text
    daily_time = row.find_all(class_='td right')[0].text
    daily_pageviews = row.find_all(class_='td right')[1].text
    temp_list.append([rank, title, daily_time, daily_pageviews])

In [64]:
# 데이터프레임으로 저장
df_alexa = pd.DataFrame(temp_list, columns=['rank', 'title', 'daily_time', 'daily_pageviews'])
df_alexa.head()

,rank,title,daily_time,daily_pageviews
0,1,Google.com,17:29,18.51
1,2,Naver.com,15:13,13.10
2,3,Youtube.com,19:33,10.57
3,4,Daum.net,12:09,8.66
4,5,Tistory.com,5:56,4.03


In [65]:
df_alexa.tail()

,rank,title,daily_time,daily_pageviews
45,46,Jobkorea.co.kr,5:51,5.70
46,47,Hankyung.com,2:16,2.54
47,48,Yna.co.kr,4:21,3.77
48,49,Inven.co.kr,6:18,6.41
49,50,Ebay.com,9:60,7.80


In [66]:
# csv 파일로 저장
df_alexa.to_csv('output2/alexa.csv', index=False)

In [68]:
ls output2

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: F424-1196

 C:\Users\User\Desktop\Code\20211118 Python Lib 4일차\output2 디렉터리

2021-11-18  오후 04:33    <DIR>          .
2021-11-18  오후 04:33    <DIR>          ..
2021-11-18  오후 04:33             1,308 alexa.csv
2021-11-18  오후 02:58            15,241 google.html
               2개 파일              16,549 바이트
               2개 디렉터리  434,981,834,752 바이트 남음
